In [2]:
import numpy
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split

import csv
import time

import os

# !ls
os.chdir('/content/drive/')

start_time = time.time()

FEATRUE_PATH = 'nn/smart_city/data_scene1.csv'
TRAIN_DATA_PATH = 'nn/smart_city/train_data/scene1_jiading_lib_training'
MODEL_PATH = 'nn/smart_city/models/scene1/scene1_model'
RESULT_PATH = 'nn/smart_city/result.csv'

TRAIN_NUMS = 10000


if __name__ == '__main__':
  # 加载数据
  data = pd.read_csv(FEATRUE_PATH)
  data.columns = ['f_x', 'f_y', 'label_x', 'label_y', 'label_z']
  features = data[['f_x', 'f_y']]
  labels = data[['label_x', 'label_y', 'label_z']]

  X = tf.placeholder(dtype=tf.float32, shape=[None, 2])
  y = tf.placeholder(dtype=tf.float32, shape=[None, 3])
  
  # 定义模型参数
  W1 = tf.Variable(tf.truncated_normal([2, 50], stddev=0.1), name='W1')
  b1 = tf.Variable(tf.zeros([50]), name='b1')
  layer_1 = tf.matmul(X, W1) + b1
  
  W2 = tf.Variable(tf.truncated_normal([50, 3], stddev=0.1), name='W2')
  b2 = tf.Variable(tf.zeros([3]), name='b2')
  _y = tf.nn.relu(tf.matmul(layer_1, W2) + b2)
  
  # 损失函数定义为 均方差
  mse_loss = tf.losses.mean_squared_error(y, _y)
  optmizer = tf.train.GradientDescentOptimizer(0.000001).minimize(mse_loss)
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(TRAIN_NUMS):
      # 使用随机梯度下降
      X_train, _, y_train, _ = train_test_split(features, labels, test_size=0.3, 
                                                random_state=42)
      sess.run(optmizer, feed_dict={X: X_train, y: y_train})
#       print('W1:\n', sess.run(W1))
#       print('b1:\n', sess.run(b1))
      
    # 保存模型
    saver = tf.train.Saver()
    save_path = saver.save(sess, MODEL_PATH)
    print('模型保存在', save_path)
    
  
  graph = tf.get_default_graph()
  W1 = graph.get_tensor_by_name('W1:0')
  b1 = graph.get_tensor_by_name('b1:0')
  W2 = graph.get_tensor_by_name('W2:0')
  b2 = graph.get_tensor_by_name('b2:0')
  
  temp = tf.matmul(tf.cast(features, tf.float32), W1) + b1
  _y = tf.matmul(tf.cast(temp, tf.float32), W2) + b2
  
  with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    _y = _y.eval()

    with open(RESULT_PATH, 'w') as f:
      writer = csv.writer(f)
      writer.writerows(_y)

    mse = ((labels - _y) ** 2).sum(axis=0).sum(axis=0) // labels.shape[0]
    print('mse =', mse)
    
    print('time = %ss'%(time.time() - start_time))
    


# 参考
# 1、按列取、按索引/行取、按特定行列取：https://www.cnblogs.com/nxf-rabbit75/p/10105271.html
 

FileNotFoundError: ignored